In [1]:
import os
foodfns = os.listdir('./foods/')

In [2]:
foodfns

['Carrot.jpg',
 'cookie.jpg',
 'orange.jpg',
 'apple.jpg',
 'pear.jpg',
 'banana.jpg']

In [3]:
from ipywidgets import interact, fixed, Image as DImage
from IPython.display import display, Image as WImage
from IPython.core.display import HTML 
wim = None
def innotater(i, path=None):
    fn = foodfns[i]
    print(fn)
    full_path = fn
    if path is not None:
        full_path = path + fn
        
    file = open(full_path, "rb")
    im = file.read()
    
    wim = WImage(im, width=150, height=100)
    #display(wim)
    
    return wim
    
    #display(DImage(filename=full_path, width=150, height=100))
#    display(wim)

In [4]:
interact(innotater, i=(0,len(foodfns)-1), path=fixed('./foods/'));

interactive(children=(IntSlider(value=2, description='i', max=5), Output()), _dom_classes=('widget-interact',)…

In [5]:
wim = innotater(2, path='./foods/')

orange.jpg


In [6]:
wim.format

'jpeg'

In [42]:
from ipywidgets import DOMWidget, HBox, VBox, IntSlider, Checkbox
from traitlets import Unicode, Int, List, Bool

class InnotaterWidget(DOMWidget):
    _view_module = Unicode('innotaterwidget').tag(sync=True)
    _view_module_version = Unicode('0.1.0').tag(sync=True)
    _view_name = Unicode('InnotaterWidgetView').tag(sync=True)
    _model_module = Unicode('innotaterwidget').tag(sync=True)
    _model_module_version = Unicode('0.1.0').tag(sync=True)
    _model_name = Unicode('InnotaterWidgetModel').tag(sync=True)
    index = Int().tag(sync=True)
    inputs = List().tag(sync=True)
    path = Unicode('').tag(sync=True)
    targets = List([0,0,0,0,0,0]).tag(sync=True)
    
    checkbox_value = Bool().tag(sync=True)


In [78]:
%%javascript
require.undef('innotaterwidget');

define('innotaterwidget', ['@jupyter-widgets/base', 'jquery', 'underscore'], function(widgets, $, _) {
    
    var defs = _.extend(widgets.DOMWidgetModel.prototype.defaults(), {
            _model_name: 'InnotaterWidgetModel',
            _view_name: 'InnotaterWidgetView',
            _model_module: 'jupyter-innotater',
            _view_module: 'jupyter-innotater',
            _model_module_version: '0.1.0',
            _view_module_version: '0.1.0',
            index: 0,
            inputs: [],
            path: '',
            targets: [0,1,0,1,0,1]
        });
    
    console.log(defs);

    var InnotaterWidgetModel = widgets.DOMWidgetModel.extend({
        defaults: defs
    });
    
    var InnotaterWidgetView = widgets.DOMWidgetView.extend({
        render: function() {
            InnotaterWidgetView.__super__.render.apply(this, arguments);
            this._count_changed();
            this.listenTo(this.model, 'change:index', this._count_changed, this);
            this.listenTo(this.model, 'change:checkbox_value', this._checkbox_value_changed, this);
            this.listenTo(this.model, 'change:targets', this._targets_changed, this);
        },

        _count_changed: function() {
            var old_value = this.model.previous('index');
            var index = this.model.get('index');
            var path = this.model.get('path');
            var inputs = this.model.get('inputs');
            var targets = this.model.get('targets').slice();
            
            var domcontainer = $('<div></div>');
            var domimage = $('<img></img>', {'src': path+inputs[index], 'width': '150', 'height': '100'});
            var domtext = $('<div></div>').append(String(index));
            
            var cur_value = targets[index];
            
            console.log(typeof(targets));
            console.log(typeof(targets[index]));
            
            this.model.set('checkbox_value', !!cur_value);
            this.model.save_changes();
            
            domcontainer.append(domimage).append(domtext);
            
            $(this.el).empty().append(domcontainer);
        },
        
        _checkbox_value_changed: function() {
            var index = this.model.get('index');
            var targets = this.model.get('targets').slice();
        
            var cur_value = !!targets[index];
            var new_value = this.model.get('checkbox_value');
            
            if (new_value != cur_value) {
                targets[index] = Number(new_value); // Need 0/1 not false/true
                console.log(targets);
                this.model.set({'targets': targets});
                console.log(this.model.get('targets'));
                this.model.save_changes();
                console.log(this.model.get('targets'));
            }
        },
        _targets_changed: function() {
        }
    });

    return {
        InnotaterWidgetView: InnotaterWidgetView,
        InnotaterWidgetModel: InnotaterWidgetModel
    }
});

<IPython.core.display.Javascript object>

In [79]:
c = Checkbox()
w = InnotaterWidget()
w.inputs = foodfns
w.targets = [1] * 6
w.path = './foods/'

s = IntSlider(min=0, max=5)
v = VBox([w,c,s])
jsl = widgets.jslink((s, 'value'), (w ,'index'))
jsl2 = widgets.jslink((c, 'value'), (w ,'checkbox_value'))


In [80]:
display(v)

In [82]:
w.targets

[0, 1, 0, 1, 1, 1]

In [41]:
w.targets = [1,0,0,1,1,0]